## Import Section

In [126]:
import pandas as pd
import numpy as np 
import time
from datetime import datetime
from datetime import date
import os
import glob
import re

## Config Section

In [127]:
#MSC
#path = 'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting'
path = 'C:/Users/Administrator/OneDrive - Griky/Documentos - Griky Dashboard/Reporting'


dataframes = []


# Udemy progress MSC
path_udemy_course_activity = f'{path}/Griky/Fuente/Catálogos/Udemy_msc/User_course_activity/'
# Udemy users MSC
path_udemy_user_activity = f'{path}/Griky/Fuente/Catálogos/Udemy_msc/User_activity/'


# Udemy progress OKTA
#path_udemy_course_activity = 'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Catálogos/Udemy_okta/User_course_activity/'

# Learnlight progress MSC
path_learnlight_progress_report= f'{path}/Griky/Fuente/Catálogos/Learnlight_msc/Progreso_report/'

#Netzun progress
path_netzun_report_full_students = f'{path}/Griky/Fuente/Catálogos/Netzun/report_full_students/'
#Netzun users



In [128]:
today = date.today()
d1 = today.strftime("%m-%d-%Y")

## Functions Section

In [129]:
def get_concatenated_files_msc(path, folder_name = 'Learner_overview', file_name= 'learner-overview', file_type = '.csv'):
    # path = Contains the path: "C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting"
    # folder_name = Name of the folder that contains the files
    # file_name = common part of the file to search
    # file_type = extension of the file: .csv, .xls, .xlsx
    dataframes = []
    for client_folder in os.listdir(path):
        path_cliente = os.path.join(path, client_folder, folder_name)
        if os.path.isdir(path_cliente):
            files = glob.glob(os.path.join(path_cliente, f'*-{file_name}-*{file_type}'))
            if files:
                # ordenar los archivos por la fecha en su nombre
                files.sort(key=lambda x: x.split(f'-{file_name}-')[-1], reverse=True)
                # abrir el archivo más reciente y cargarlo en un dataframe
                df = pd.read_csv(files[0])
                df = df.dropna(axis=0, how='all')
                df = df.dropna(axis=1, how='all')
                df['Cliente'] = client_folder
                # extract the date from the filename
                date_str = files[0].split(f'-{file_name}-')[1].split('-')[0:3]
                date_str = '-'.join(date_str)
                date = datetime.strptime(date_str, "%Y-%m-%d")
                df['fecha_archivo'] = date.date()
                # agregar el dataframe a la lista
                dataframes.append(df)
    df_final = pd.concat(dataframes)
    return df_final

In [130]:
def get_last_file(path, file_name, file_type, date_format = "%Y-%m-%d"):
    # Buscar todos los archivos que coinciden con el patrón en el directorio
    files = glob.glob(os.path.join(path, f'{file_name}_*{file_type}'))
    
    if not files:
        return None     # No se encontraron archivos
    
    # Ordenar los archivos por fecha y seleccionar el último
    files.sort(key=lambda x: datetime.strptime(x.split(f'{file_name}_')[-1].split('.')[0], date_format), reverse=True)
    
    # Lee el archivo en un pandas DataFrame
    if file_type == '.csv':
        df = pd.read_csv(files[0])
    elif file_type in ['.xlsx', '.xls']:
        df = pd.read_excel(files[0])
    else:
        print('File type not supported')
        return None

    return df  # Devuelve el DataFrame
    

In [131]:
def get_last_file_new(path, file_type='.xlsx', date_format="%Y-%m-%d", regex_pattern=None):
    all_files = glob.glob(os.path.join(path, f'*{file_type}'))
    
    # Check if any files were found
    if not all_files:
        return None
    
    if regex_pattern:
        date_files = []
        for file in all_files:
            dates = re.findall(regex_pattern, file)
            if dates:
                last_date = max(datetime.strptime(date, date_format) for date in dates)
                date_files.append((last_date, file))
        
        # Sort the files by date
        date_files.sort(reverse=True)
        latest_file = date_files[0][1]
        print("Latest file date is: ", date_files[0][0])
    else:
        latest_file = max(all_files, key=os.path.getctime)

    # Read the file into a DataFrame
    if file_type == '.csv':
        df = pd.read_csv(latest_file)
    elif file_type in ['.xlsx', '.xls']:
        df = pd.read_excel(latest_file)
    else:
        print('File type not supported')
        return None

    return df

## Get Dataframes

### Progreso dataframes

In [132]:
df_progreso_msc = get_concatenated_files_msc(path = path, folder_name = 'Learner_engagement', file_name= 'learner-engagement')
df_progreso_msc.head(1)

,ID Learner,External ID Learner,Last Name,First Name,Email Address,Job Title,Department,City,Manager,Account type,...,Progress (%),Learning Time (min),Score (%),Enrolment Date,Start Date,Completion Date,Cliente,fecha_archivo,Certified,Certificate URL
0,243406.0,NaN,Ochoa Rodríguez,Juan Gabriel,juan.ochoa@griky.co,NaN,Griky,NaN,NaN,user,...,0.0,0.0,0.0,2023-05-29 15:11:10,2023-05-29 15:11:39,NaN,Alata,2023-11-17,NaN,NaN


In [133]:
#df_progreso_udemy = get_last_file(path_udemy_course_activity, file_name= 'user_course_activity_report_griky-myskillcamp', file_type='.csv')
df_progreso_udemy = get_last_file_new(path_udemy_course_activity, file_type='.csv', date_format= "%Y-%m-%d", regex_pattern= r'\d{4}-\d{2}-\d{2}')
df_progreso_udemy.sample(10).head(1)

Latest file date is:  2023-11-14 00:00:00


,User First Name,User Last Name,User Email,External ID,User Groups,Course ID,Course Title,Course Locale,Course Duration,% Marked Completed,Minutes Video Consumed,Date Enrolled,Date Started,First Date Completed,Date Completed,Date Last Active,Course Category,Assigned,Assigned By,User Is Deactivated
12242,Andrés,Coronel,andres.coronel@uees.edu.ec,NaN,NaN,1379334,Internet de las Cosas: una introducción al mun...,es_ES,354.0,100.0,132.25,2023-09-29 00:09:16+00:00,2023-09-30 19:42:07+00:00,2023-10-26 21:17:42+00:00,2023-10-26 21:17:42+00:00,2023-10-26,Hardware,False,NaN,No


In [134]:
df_progreso_netzun = get_last_file_new(path_netzun_report_full_students, file_type='.xlsx', 
                               date_format="%m-%d-%Y", 
                               regex_pattern=r'\d{2}-\d{2}-\d{4}')
df_progreso_netzun.sample(10).head(1)

Latest file date is:  2023-11-17 00:00:00


,ID,Nombres y Apellidos,Email,Función,Curso,Inscripción del curso,Estado de curso,Lecciones Tomadas/Total Lecciones,Porcentaje de Progreso,Tiempo total,Puntaje Porcentaje,Fecha última lección,Fecha Deadline,Áreas,Código Interno,Filtro
5233,784565,Astrid Carolina Lopez Palta,lopez.astrid@fuac.edu.co,Usuario,Content Marketing,15/06/2023,Completado con Certificado,12/12,100%,2:13:18,80/100,15/06/2023,-,Autónoma,NaN,NaN


In [135]:
df_progreso_learnlight = get_last_file_new(path_learnlight_progress_report, file_type='.xlsx', 
    date_format= "%Y-%m-%d", 
    regex_pattern=r'\d{4}-\d{2}-\d{2}')
df_progreso_learnlight = df_progreso_learnlight.drop([0,1])
df_progreso_learnlight.columns = df_progreso_learnlight.iloc[0]
df_progreso_learnlight =df_progreso_learnlight[1:]
df_progreso_learnlight = df_progreso_learnlight.reset_index(drop=True)
df_progreso_learnlight.head(5)

Latest file date is:  2023-11-08 00:00:00


2,Nombre,Apellido,Usuario,Correo electrónico,Grupo actual,Asignatura,Servicio,Complementos,Curso actual,Progreso del curso actual,...,Puesto de trabajo,Director,Departamento,Entidad operativa,Entidad de facturación,País,Ciudad,Fecha de inicio del programa,Fecha de fin del programa,Estado del Perfil
0,Abigail,Serrano,abigailzoquiapa43@gmail.com,abigailzoquiapa43@gmail.com,-,English,Digital: Standard,Assessment: Placement,A1 - Beginner 1,0.083,...,NaN,-,-,-,UGC Más,Mexico,Los cabos,20-09-2023,18-09-2024,Habilitado
1,Abran,Sánchez,abram4423@gmail.com,abram4423@gmail.com,-,English,Digital: Standard,Assessment: Placement,A1 - Beginner 1,0.087,...,NaN,-,-,-,UGC Más,Mexico,Cabo san lucas,27-10-2023,25-10-2024,Habilitado
2,Abran,Sánchez,abram4423@gmail.com,abram4423@gmail.com,-,Italiano,Digital: Standard,Assessment: Placement,A1 - Principiante 1,0,...,NaN,-,-,-,UGC Más,Mexico,Cabo san lucas,14-10-2023,12-10-2024,Habilitado
3,Ada,Camila,a.diaz57@alumnos.santotomas.cl,a.diaz57@alumnos.santotomas.cl,-,English,Digital: Standard,Assessment: Placement,A1 - Beginner 1,0,...,NaN,-,-,-,SantoTomas,Chile,Puerto Montt,16-09-2023,14-09-2024,Habilitado
4,Ada Estela,Colato Reyes,acolato@ugb.edu.sv,acolato@ugb.edu.sv,-,English,Digital: Standard,Assessment: Placement,A1 - Beginner 1,0,...,NaN,-,-,-,Ugb,El Salvador,San Miguel,14-08-2023,12-08-2024,Habilitado


### Usuarios Dataframes

In [136]:
df_usuarios_msc = get_concatenated_files_msc(path = path, folder_name = 'Learner_overview', file_name= 'learner-overview')
df_usuarios_msc.sample(1).head(1)

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_10832\3943988448.py:15: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(files[0])
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_10832\3943988448.py:15: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(files[0])


,ID Learner,External ID Learner,Last Name,First Name,Email Address,Brand Info,Job Title,Department,City,Manager,...,Activation Date,Last connection date,Camp Access,Registered to a content,Learning time spent (min),Content registrations,Content completions,Certifications obtained,Cliente,fecha_archivo
63458,491728.0,leandrowilliman@hotmail.com,Williman,Leandro Matias,leandrowilliman@hotmail.com,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"Cegos - Habilidades Humanas, Griky - Habilidad...",N,0.0,0.0,0.0,0.0,Jerarquicos,2023-11-02


In [137]:
df_usuarios_udemy = get_last_file(path_udemy_user_activity, file_name= 'user_activity_report_griky-myskillcamp', file_type='.csv')
df_usuarios_udemy.sample(3)

,User First Name,User Last Name,User Email,External ID,User Groups,User Role,User Is Deactivated,Date Joined,Date of Last Activity,No. of Days Active,No. of New Courses Enrolled,No. of New Courses Started,No. of Courses Completed,Minutes Video Consumed,No. of Lectures Completed,No. of Assigned Courses,No. of Practice Activities Completed
4687,NaN,NaN,trejojacqueline28@gmail.com,NaN,NaN,student,No,2023-10-29,2023-11-09,2,2,2,2,255.50,66,0,7
3208,NaN,NaN,delacruzrubyoli@gmail.com,NaN,NaN,student,No,2023-09-25,2023-10-27,12,2,2,0,180.75,55,0,3
2351,NaN,NaN,t_orellana@ugb.edu.sv,NaN,NaN,student,No,2023-08-28,2023-08-28,1,1,0,0,0.00,0,0,0


## Modify Data Frames

### Config dict

In [138]:
rename_dic_progreso_udemy = {
    #“old_column_name” : “new_column_name”
    'User First Name' : 'First Name', 
    'User Last Name' : 'Last Name', 
    'User Email' : 'Email Address', 
    #'External ID' : '',
    #'User Groups', 
    'Course ID' : 'Content ID' , 
    'Course Title' : 'Content Title', 
    #'Course Locale',
    'Course Duration' : 'Estimated course duration (min)', #revisar unidades
    '% Marked Completed' : 'Progress (%)', #modify units
    'Minutes Video Consumed' : 'Learning Time (min)', #modify units
    'Date Enrolled' : 'Enrolment Date', #Modify dates format
    'Date Started' : 'Start Date',  #Modify dates format
    #'First Date Completed',
    'Date Completed' : 'Completion Date',  #Modify dates format
    #'Date Last Active', 
    'Course Category' : 'Categories',  
    #'Assigned',
    #'Assigned By', 
    #'User Is Deactivated'


}


rename_dic_progreso_netzun = {
    #“old_column_name” : “new_column_name”
    
    "ID" : "ID Learner",
    'Nombres y Apellidos' : 'Name',
    'Email' : 'Email Address', 
    #'Función' : '', 
    'Curso' : 'Content Title',
    'Inscripción del curso' :  'Enrolment Date', 
    'Estado de curso' : 'Status',
    #'Lecciones Tomadas/Total Lecciones', 
    'Porcentaje de Progreso' : 'Progress (%)' ,  #Modificar unidades
    'Tiempo total' : 'Learning Time (min)' ,    # Pasar a minutos
    #'Puntaje Porcentaje', 
    'Fecha última lección' : 'Date Last Active',
    #'Fecha Deadline', 
    #'Áreas' : 'Cliente',  #Modificar nombres para que coincida 
    #'Código Interno', 
    #'Filtro'
}

rename_dic_progreso_learnlight = {
    #“old_column_name” : “new_column_name”

    'Nombre': 'First Name' , 
    'Apellido' : 'Last Name', 
    'Correo electrónico' : 'Email Address' , 
    'Asignatura' : 'Content Title', 
    #'Servicio', 
    #'Complementos', 
    'Curso actual' : 'Content Title_',
    'Progreso del curso actual' : 'Progress (%)', 
    #'Tiempo de conexión',
    'Tiempo de estudio total (HH:mm)' : 'Learning Time (min)',
    # 'Tiempo actividades interactivas',
    # 'Total de actividades completadas',
    # 'Total de actividades - puntuación media', 
    # 'Examen de progreso 1',
    # 'Examen de progreso 2', 'Examen de progreso 3',
    # 'Examen de progreso - puntuación media',
    # 'Actividades autoestudio del itinerario pedagógico',
    # 'Actividades autoestudio del itinerario pedagógico - puntuación media',
    # 'Actividades autoestudio personalizadas',
    # 'Actividades autoestudio personalizadas - puntuación media',
    # 'Actividades de la Revista',
    # 'Actividades de la Revista - puntuación media',
    # 'Actividades realizadas en sesiones con formador',
    # 'Actividades sesiones Café', 'Prueba final', 'Socio', 'Empresa',
    # 'Puesto de trabajo', 'Director', 'Departamento', 'Entidad operativa',
    #'Entidad de facturación' : 'Cliente', 
    # 'País', 'Ciudad',
    'Fecha de inicio del programa' : 'Enrolment Date', 
    #'Fecha de fin del programa',
    #'Estado del Perfil'
}

final_feautres_dic_progreso_udemy = [

    'Email Address', 
    'Content ID' , 
    'Content Title', 
    'Estimated course duration (min)', #revisar unidades
    'Progress (%)', #modify units
    'Status',
    'Learning Time (min)', #modify units
    'Enrolment Date', #Modify dates format
    'Start Date',  #Modify dates format
    'Completion Date',
    'Date Last Active',
    'Categories',
    'Cliente',
    'Catalogo'
]

final_feautres_dic_progreso_netzun = [
    #“old_column_name” : “new_column_name”
    
    "ID Learner",
    'Name',
    'Email Address', 
    'Content Title',
    'Enrolment Date', 
    'Completion Date',
    'Date Last Active',
    'Status',
    'Progress (%)' ,  #Modificar unidades
    'Learning Time (min)' ,    # Pasar a minutos
    'Cliente',  #Modificar nombres para que coincida 
    'Catalogo'
]

final_feautres_dic_progreso_learnlight = [
    #“old_column_name” : “new_column_name”

    'Name',
    'Email Address', 
    'Content Title',
    'Enrolment Date', 
    #'Completion Date',
    #'Date Last Active',
    'Status',
    'Progress (%)' ,  #Modificar unidades
    'Learning Time (min)' ,    # Pasar a minutos
    'Cliente',  #Modificar nombres para que coincida 
    'Catalogo'
]

final_feautres_dic_progreso_msc = [
    #'ID Learner',
    'Email Address',
    'Content ID', 
    'Content Type',
    #'Training Type', 
    'Content Title', 
    #'Session Title', 
    'Categories', 
    'Camps Published',
    'Content Provider', 
    'Creator', 
    'Trainer(s)', 
    'Creation date',  #Course creation date
    'Last update',    #Course last update
    'Skills', 
    'Language', 
    'Estimated course duration (min)',
    'Status', 
    'Progress (%)', 
    'Learning Time (min)', 
    'Score (%)',
    'Enrolment Date', 
    'Start Date', 
    'Completion Date', 
    'Cliente',
    'Name',
    'Catalogo',
    'Certified',
    'Certificate URL'
    
]
    
    

In [139]:
rename_dic_progreso_netzun_cliente = {
    'ALATA' : 'Alata',
    'Anahuac' : 'Anahuac',
    'Autónoma' : 'Autonoma',
    'BD Guidance' : 'Bdguidance',
    'Buencafé' : 'Buencafe',
    'CCB' : 'Ccb',
    'CEIPA' : 'Ceipa',
    'Class-run' : 'Classrun', 
    'Colsubsidio' :  'Colsubsidio',   #REVISAR
    'Corona' : 'Corona',
    'DALTA' : 'Dalta',
    'EANX' : 'Eanx', 
    'EDP' : 'Edp',
    'Eficacia' : 'Eficacia',
    'EIG' : 'Globali',
    'FinT Coach' : 'FinT Coach',   #REVISAR
    'Fundación Neón' : 'Holastaff',
    'Griky' : 'Griky',
    'INCAP' : 'Incap',
    'Incurso' : 'Incurso',
    'INMED' : 'Inmed',
    'Insurgentes' : 'Uinsurgentes',
    'ISTEONLINE' : 'ISTE',        #REVISAR
    'Jerarquicos' : 'Jerarquicos',
    'POLI' : 'Poli',
    'RDITS' : 'RD_ITS',
    'SECORSE' : 'Secorse',
    'Streaming Learning' : 'Streaminglearning',
    'SURA' : 'Sura',
    'Sysneo' : 'Central+',
    'Teleperformance' : 'Teleperformance',
    'UCPx' : 'Ucpx',
    'UDES' : 'Udes',
    'UGB' : 'Ugb',
    'UGC' : 'Ugc',
    'UMA' : 'Uma',
    'UMB' : 'Umb',
    'UNE' : 'Une',
    'Uniagustiniana' : 'UniAgustiniana',
    'UNIBE' : 'Unibe',
    'UNIR' : 'Unir', 
    'UPARSISTEM' : 'Uparsistem',
    'USAP' : 'Usap',
    'UST Plus' : 'SantoTomas',
    'UTESA' : 'Utesa', 
    'UTMA' : 'Utma',
    'WURTH' : 'Wurth'
    
    
}

### Usuarios MSC

In [140]:
df_usuarios_msc = df_usuarios_msc[~((df_usuarios_msc['Cliente'] != "Griky" )  & (df_usuarios_msc['Email Address'].str.contains('griky')))]


In [141]:
usuarios_griky = df_usuarios_msc[df_usuarios_msc['Cliente'] == 'Griky']['Email Address']
not_griky = df_usuarios_msc['Cliente'] != 'Griky'
in_list = df_usuarios_msc['Email Address'].isin(usuarios_griky)
filter_ = not_griky & in_list
df_usuarios_msc = df_usuarios_msc[~filter_]
duplicated_df = df_usuarios_msc[df_usuarios_msc.duplicated(subset='Email Address', keep=False)]
df_usuarios_msc = df_usuarios_msc.drop_duplicates(subset='Email Address', keep='last')
#df_usuarios_msc[ 'Email Address'] = df_usuarios_msc['Email Address'].str.lower().str.strip()
df_usuarios_msc['Email Address'] = df_usuarios_msc['Email Address'].str.lower().str.strip()  #.str.replace('[^a-zA-Z@.]', '', regex=True)
df_usuarios_msc['Activation Date'] = pd.to_datetime(df_usuarios_msc['Activation Date']).dt.normalize()
df_usuarios_msc['Last connection date'] = pd.to_datetime(df_usuarios_msc['Last connection date']).dt.normalize()

### Usuarios Udemy MSC

In [142]:
df_usuarios_udemy.head(1)

,User First Name,User Last Name,User Email,External ID,User Groups,User Role,User Is Deactivated,Date Joined,Date of Last Activity,No. of Days Active,No. of New Courses Enrolled,No. of New Courses Started,No. of Courses Completed,Minutes Video Consumed,No. of Lectures Completed,No. of Assigned Courses,No. of Practice Activities Completed
0,Rafael,Rivas,rafael.rivas@griky.co,NaN,NaN,owner,No,2022-11-04,2023-11-10,5,6,3,0,2.75,2,0,0


In [143]:
df_usuarios_udemy.shape

(5200, 17)

In [144]:
df_usuarios_udemy = df_usuarios_udemy.merge(df_usuarios_msc[['Email Address', 'Cliente']], left_on='User Email', right_on= 'Email Address' ,how='left')


### Progreso_msc

In [145]:
df_progreso_msc['Name'] = df_progreso_msc['First Name'] + " " + df_progreso_msc['Last Name']
df_progreso_msc['Camps Published'] = df_progreso_msc['Camps Published'].str.lower()
df_progreso_msc = df_progreso_msc[df_progreso_msc['Content Type'] != 'External content']
condition_external_catalog = (
        (df_progreso_msc['Content Type'] == 'Pinned content') 
        & 
        (
            (df_progreso_msc['Camps Published'].str.contains('netzun')) | (df_progreso_msc['Camps Published'].str.contains('udemy'))
        )
    )

df_progreso_msc = df_progreso_msc[ ~condition_external_catalog]
df_progreso_msc['Catalogo'] = 'MSC'
df_progreso_msc['Status'] = df_progreso_msc['Status'].replace({
                                                                'Not started' : 'Not Started',
                                                                'In progress' : 'In Progress'
                                                               })
df_progreso_msc['Email Address'] = df_progreso_msc['Email Address'].str.lower().str.strip() #.str.replace('[^a-zA-Z@.]', '', regex=True)
df_progreso_msc['Progress (%)'] = df_progreso_msc['Progress (%)'].fillna(np.nan).astype('Int16')
df_progreso_msc['Enrolment Date'] = pd.to_datetime(df_progreso_msc['Enrolment Date']).dt.normalize()
df_progreso_msc['Completion Date'] = pd.to_datetime(df_progreso_msc['Completion Date']).dt.normalize()
df_progreso_msc['Start Date'] = pd.to_datetime(df_progreso_msc['Start Date']).dt.normalize()
df_progreso_msc['Creation date'] = pd.to_datetime(df_progreso_msc['Creation date']).dt.normalize()
df_progreso_msc['Last update'] = pd.to_datetime(df_progreso_msc['Last update']).dt.normalize()
df_progreso_msc = df_progreso_msc[final_feautres_dic_progreso_msc]

### Progreso_udemy

In [146]:
df_progreso_udemy.rename(columns=rename_dic_progreso_udemy, inplace=True)
df_progreso_udemy['Name'] = df_progreso_udemy['First Name'] + " " + df_progreso_udemy['Last Name'] 
df_progreso_udemy['Email Address'] = df_progreso_udemy['Email Address'].str.lower().str.strip() #.str.replace('[^a-zA-Z@.]', '', regex=True)
df_progreso_udemy = df_progreso_udemy.merge(df_usuarios_msc[['Email Address', 'Cliente']], on='Email Address', how='left')
df_progreso_udemy['Catalogo'] = 'Udemy'
#df['Status'] = df['Progress (%)'].apply(lambda x: 'Completed' if x == 100 else ('Not Started' if pd.isnull(x) or x==0 else 'In Progress'))
df_progreso_udemy['Status'] = df_progreso_udemy['Progress (%)'].apply( lambda x: 'Completed' if x == 100 else ('Not Started' if pd.isnull(x) or x == 0 else 'In Progress'))
df_progreso_udemy['Progress (%)'] = df_progreso_udemy['Progress (%)'].fillna(np.nan).astype('Int16')
df_progreso_udemy['Enrolment Date'] = pd.to_datetime(df_progreso_udemy['Enrolment Date'], utc=True).dt.tz_localize(None).dt.normalize()
df_progreso_udemy['Start Date'] = pd.to_datetime(df_progreso_udemy['Start Date'], utc=True).dt.tz_localize(None).dt.normalize()
df_progreso_udemy['Completion Date'] = pd.to_datetime(df_progreso_udemy['Completion Date'], utc=True).dt.tz_localize(None).dt.normalize()
df_progreso_udemy['Date Last Active'] = pd.to_datetime(df_progreso_udemy['Date Last Active'], utc=True).dt.tz_localize(None).dt.normalize()
df_progreso_udemy = df_progreso_udemy[final_feautres_dic_progreso_udemy]


### Progreso_netzun

In [147]:
def convert_to_minutes(time_string):
    h, m, s = map(int, time_string.split(':'))
    return h*60 + m + s/60

In [148]:
df_progreso_netzun = df_progreso_netzun.merge(df_usuarios_msc[['Email Address','Cliente']], 
                                              left_on='Email', right_on='Email Address', how='left')
df_progreso_netzun['Áreas'] = df_progreso_netzun['Áreas'].replace(rename_dic_progreso_netzun_cliente)
df_progreso_netzun['Cliente'] = df_progreso_netzun['Áreas'].fillna(df_progreso_netzun['Cliente'])
df_progreso_netzun = df_progreso_netzun.drop(['Email Address', 'Áreas'], axis=1)
df_progreso_netzun.rename(columns=rename_dic_progreso_netzun, inplace=True)

df_progreso_netzun['Catalogo'] = 'Netzun'
#df_progreso_netzun['Status'] = df_progreso_netzun['Status'].apply(lambda x: 'Completed' if x == 'Completado con Certificado' or x == 'Completado' else ('Not Started' if x == 'No Iniciado' else 'In Progress' if x == 'En Progreso' else "") )
df_progreso_netzun['Status'] = df_progreso_netzun['Status'].replace({
                                                                        'Completado con Certificado' : 'Completed',
                                                                        'Completado' : 'Completed',
                                                                        'No Iniciado' : 'Not Started',
                                                                        'En Progreso' : 'In Progress' })
df_progreso_netzun['Progress (%)'] = df_progreso_netzun['Progress (%)'].str.replace('%', '').astype('Int16') 
#df_progreso_netzun['Progress (%)'] = df_progreso_netzun['Progress (%)'].str.replace('%', '').str.strip().replace('', np.nan).astype('Int16')
df_progreso_netzun['Learning Time (min)'] = df_progreso_netzun['Learning Time (min)'].apply(convert_to_minutes)
df_progreso_netzun['Date Last Active'] = df_progreso_netzun['Date Last Active'].replace('-', np.nan)
df_progreso_netzun['Completion Date'] = df_progreso_netzun.apply(
    lambda row: row['Date Last Active'] if row['Progress (%)'] == 100 else '', axis=1)
df_progreso_netzun['Email Address'] = df_progreso_netzun['Email Address'].str.lower().str.strip() #.str.replace('[^a-zA-Z@.]', '', regex=True)
df_progreso_netzun['Enrolment Date'] = pd.to_datetime(df_progreso_netzun['Enrolment Date'], format='%d/%m/%Y')
df_progreso_netzun['Completion Date'] = pd.to_datetime(df_progreso_netzun['Completion Date'], format='%d/%m/%Y', errors='coerce')
                                        # pd.to_datetime(df_progreso_consolidado['Date Last Active'], format='%Y/%m/%d', errors='coerce')
# df_progreso_netzun['Completion Date'] = np.where(df_progreso_netzun['Status'] == 'Completed', 
#                                                  pd.to_datetime(df_progreso_netzun['Date Last Active'], format='%d/%m/%Y'), 
#                                                  np.nan)
df_progreso_netzun = df_progreso_netzun[final_feautres_dic_progreso_netzun]


### Progreso Learlight

In [150]:
def convert_to_minutes_(time_string):
    if isinstance(time_string, str):
        h, m = map(int, time_string.split(':'))
        return h*60 + m
    else:
        return time_string  # o cualquier otro valor por defecto para datos no válidos

In [151]:

df_progreso_learnlight.rename(columns=rename_dic_progreso_learnlight, inplace=True)
df_progreso_learnlight['Name'] = df_progreso_learnlight['First Name'] + " " + df_progreso_learnlight['Last Name'] 
df_progreso_learnlight['Content Title'] = df_progreso_learnlight['Content Title'] + "-" + df_progreso_learnlight['Content Title_'] 
df_progreso_learnlight['Email Address'] = df_progreso_learnlight['Email Address'].str.lower().str.strip() #.str.replace('[^a-zA-Z@.]', '', regex=True)
df_progreso_learnlight = df_progreso_learnlight.merge(df_usuarios_msc[['Email Address', 'Cliente']], on='Email Address', how='left')
df_progreso_learnlight['Catalogo'] = 'Learnlight'
df_progreso_learnlight['Progress (%)'] = df_progreso_learnlight['Progress (%)'].replace([np.inf, -np.inf], np.nan).fillna(0)
df_progreso_learnlight['Progress (%)'] = (df_progreso_learnlight['Progress (%)']*100).round().astype(int)
df_progreso_learnlight['Status'] = df_progreso_learnlight['Progress (%)'].apply( lambda x: 'Completed' if x == 100 else ('Not Started' if pd.isnull(x) or x == 0 else 'In Progress'))
df_progreso_learnlight['Learning Time (min)'] = df_progreso_learnlight['Learning Time (min)'].replace([np.inf, -np.inf, '-'], np.nan).fillna(0)
df_progreso_learnlight['Learning Time (min)'] = df_progreso_learnlight['Learning Time (min)'].apply(convert_to_minutes_)
df_progreso_learnlight['Enrolment Date'] = pd.to_datetime(df_progreso_learnlight['Enrolment Date'], format='%d-%m-%Y')
df_progreso_learnlight = df_progreso_learnlight[final_feautres_dic_progreso_learnlight]


## Create Progress DataFrame

In [152]:
df_progreso_consolidado = pd.concat([df_progreso_msc, df_progreso_udemy,df_progreso_netzun, df_progreso_learnlight], ignore_index=True)
df_progreso_consolidado['Date Last Active'] = pd.to_datetime(df_progreso_consolidado['Date Last Active'], format='%Y/%m/%d', errors='coerce')
df_progreso_consolidado['Date Last Active'] = df_progreso_consolidado['Date Last Active'].dt.date
df_progreso_consolidado['Creation date'] =df_progreso_consolidado['Creation date'].dt.date
df_progreso_consolidado['Last update'] =df_progreso_consolidado['Last update'].dt.date
df_progreso_consolidado['Enrolment Date'] =df_progreso_consolidado['Enrolment Date'].dt.date
df_progreso_consolidado['Start Date'] =df_progreso_consolidado['Start Date'].dt.date
#df_progreso_consolidado['Completion Date'] = pd.to_datetime(df_progreso_consolidado['Completion Date'], format='%Y/%m/%d', errors='coerce')
df_progreso_consolidado['Completion Date'] =df_progreso_consolidado['Completion Date'].dt.date
df_progreso_consolidado.head(1)

,Email Address,Content ID,Content Type,Content Title,Categories,Camps Published,Content Provider,Creator,Trainer(s),Creation date,...,Learning Time (min),Score (%),Enrolment Date,Start Date,Completion Date,Cliente,Name,Catalogo,Date Last Active,ID Learner
0,juan.ochoa@griky.co,736231.0,Course,Patrimonio Turístico,Habilidades Profesionales,griky - catálogo completo,Griky,Lina Fernanda Romero Lara,NaN,2023-05-24,...,0.0,0.0,2023-05-29,2023-05-29,NaT,Alata,Juan Gabriel Ochoa Rodríguez,MSC,NaT,NaN


## Save 

In [153]:
#df_usuarios_udemy.to_excel('usuarios_udemy.xlsx', index=False)
#df_progreso_msc.to_excel(f'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Consolidado_Learner_Engagement/Progreso_MSC_{d1}.xlsx', index=False)

df_progreso_consolidado.to_excel(f'{path}/Griky/Fuente/Catálogos/4.0 - Consolidado/Historico_Progreso/Progreso_MSC_{d1}.xlsx', index=False)
df_usuarios_msc.to_excel(f'{path}/Griky/Fuente/Catálogos/4.0 - Consolidado/Historico_Usuarios/Usuarios_MSC_{d1}.xlsx', index=False)
# df_progreso_consolidado.to_excel(f'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Catálogos/4.0 - Consolidado/Progreso_MSC.xlsx', index=False)
# df_usuarios_msc.to_excel(f'C:/Users/evanm/Documents/Griky_data/OneDrive - Griky/Documentos - Griky Dashboard/Reporting/Griky/Fuente/Catálogos/4.0 - Consolidado/Usuarios_MSC.xlsx', index=False)
